In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Creation of compute cluster
# Using vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster-project-pipeline"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

**Create a project directory**
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. 
This includes the training script and any additional files your training script depends on.

In [ ]:
import os

project_folder = './bankmarketing'
os.makedirs(project_folder, exist_ok=True)

** Copy train.py script to project folder **

In [ ]:
import shutil

shutil.copy('train.py', project_folder)

** Create experiment **

In [ ]:
from azureml.core import Experiment

experiment_name = 'train_bankmarketing'
experiment = Experiment(ws, name=experiment_name)

**Create an environment**

Define a conda environment YAML file with your training script dependencies and create an Azure ML environment.

In [ ]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

In [ ]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')


** Configure the training job **

Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on.

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      arguments=['--C', 1, '--max_iter', 100, '--solver', 'lbfgs'],
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

**Submit job**

Run your experiment by submitting your ScriptRunConfig object. Note that this call is asynchronous.

In [ ]:
run = experiment.submit(src)

**Monitor your run**

You can monitor the progress of the run with a Jupyter widget. 
Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
    "--C": choice(0.01, 0.1, 1, 10, 100),
    "--max_iter": choice(100, 200, 500),
    "--solver": choice('newton-cg', 'lbfgs', 'liblinear'),    
    }
)


# Specify a Policy
# The BanditPolicy basically states to check the job every 2 iterations. 
# If the primary metric (defined later) falls outside of the top 10% range, Azure ML 
# terminate the job. This saves us from continuing to explore hyperparameters that don't 
# show promise of helping reach our target metric.
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###
# I decided not to use the estimator because the documentation (https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.sklearn.sklearn) says that it is deprecated

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

**Monitor HyperDrive runs**

Monitor the progress of the runs with the following Jupyter widget.

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
assert(hyperdrive_run.get_status() == "Completed")

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###